# Spinor connection, Dirac equation and Wald solution

## Basic initialization of the lorentzian manifold

Initialization of the Kerr manifold $\mathcal M_{Kerr}$ atlas with Boyer-Lindquist chart (BL) and of the metric tensor $g$ :

In [1]:
%display latex

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") #boyer-lindquist chart definition
#SD.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")

#var('m', domain='real')
var('m, a', domain='real') #definition of Kerr metric parameters

g = M.metric() #we put the label g on the metric tensor

rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2

g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
#g[0, 0] = - (1 - 2*m/r)
#g[1, 1] = 1/(1 - 2*m/r)
#g[2, 2] = r^2
#g[3, 3] = r^2*sin(th)^2

g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt*dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph*dph

Definition of the Levi Civita connection $\nabla$:

In [2]:
nabla = g.connection()

## Initialization of the orthonormal tetrad

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ :

In [3]:
#definition of the coframe tetrad fields (as listed in Wald's article)

de0 = M.one_form(sqrt(Delta/rho2), 0, 0, -a*(sin(th)^2)*sqrt(Delta/rho2), frame=BL.frame(), name='e^{(0)}')
de1 = M.one_form(0, sqrt(rho2/Delta), 0, 0, frame=BL.frame(), name='e^{(1)}')
de2 = M.one_form(0, 0, sqrt(rho2), 0, frame=BL.frame(), name='e^{(2)}')
de3 = M.one_form(-(sin(th)/sqrt(rho2))*a, 0, 0, (sin(th)/sqrt(rho2))*(r^2+a^2), frame=BL.frame(), name='e^{(3)}')

#obtaining the tetrad frame vector fields using musical isomorphism of the metric

e0 = -de0.up(g) #the zero component must be multiplied by a minus because of the lowering of the tetrad index with eta_{(a)(b)}
e1 = de1.up(g)
e2 = de2.up(g)
e3 = de3.up(g)


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}  

Orthonormality check $g(e_{(a)},e_{(b)}) = \eta_{(a)(b)}$ :

In [4]:
from sage.tensor.modules.comp import Components

scalar_field_algebra = M.scalar_field_algebra()

#definition of eta_{(a)(b)}
eta = Components(scalar_field_algebra, e, 2)
eta[0,0] = M.scalar_field({BL: -1}, name='-1')
eta[1,1] = M.scalar_field({BL: 1}, name='1')
eta[2,2] = M.scalar_field({BL: 1}, name='1')
eta[3,3] = M.scalar_field({BL: 1}, name='1')

for i in range(0,4):
    for j in range(0,4):
        if (g(e[i],e[j]) != eta[i,j]):
            print("error")

Check that the one forms given are equal to the one computed by Sage:

In [5]:
if (de[0] != de0 or de[1] != de1 or de[2] != de2 or de[3] != de3):
    print("error")

## Ricci rotation coefficents

Ricci rotation coefficents $\omega^{(a)}_{\ \ \ \ (b)(c)} = e^{(a)}_{\ \ \ \ \nu}\nabla_{\mu} e_{(b)}^{\ \ \ \ \nu} e_{(c)}^{\ \ \ \ \mu}$ :

In [6]:
#Ricci rotation coefficents as a class
class Ricci_rotation_coefficents:
    def __init__(self,Gamma):
        self.coef = Gamma
        self.tindices = ["up","down","down"]
        
    def __repr__(self):
        return self.coef[:]
    
    def copy(self):
        copy_ = Ricci_rotation_coefficents(self.coef.copy())
        copy_.tindices = list(self.tindices)
        return copy_
    
    def info(self):
        return f"Indices position: {self.tindices}"
        
    #def __getitem__(self,*indices):
     #   return self.coef[indices]
        
    def contract_with_eta(self,omega_,eta_,pos_index_,typ):
        if(pos_index_ == 1):
                omega_.coef = self.coef.contract(0,eta,0) #the contraction results in omega^{a}_{bc}*eta_{ad}=obj_{bcd} -> indices has to be permutated in dbc to have the correct form
                omega_.coef = omega_.coef.swap_adjacent_indices(0,2,3) #scambia (bc) con (d)
                omega_.tindices[0] = typ
        elif(pos_index_ == 2):
                omega_.coef = self.coef.contract(1,eta,0) #the contraction results in omega_{a}^{b}_{c}*eta_{bd}=obj_{acd} -> indices has to be permutated in dbc to have the correct form
                omega_.coef = omega_.coef.swap_adjacent_indices(1,2,3) #swaps (c) with (d)
                omega_.tindices[1] = typ
        elif(pos_index_ == 3):
                omega_.coef = self.coef.contract(2,eta,0) #the contraction results in omega_{ab}^{c}*eta_{cd}=obj_{abd} -> indices are already ok
                omega_.tindices[2] = typ
        
    def down(self,pos_index):
        if (self.tindices[pos_index-1] == "down"):
            print("error, index already down")
        else:
            omega_down = self.copy()
            self.contract_with_eta(omega_down,eta,pos_index,"down")
            return omega_down
        
    def up(self,pos_index):
        if (self.tindices[pos_index-1] == "up"):
            print("error, index already up")
        else:
            omega_up = self.copy()
            self.contract_with_eta(omega_up,eta,pos_index,"up")
            return omega_up
        

Calculation of $\omega^{(a)}_{\ \ \ \ (b)(c)}$ from $D(e_{(c)},e_{(b)})=\omega^{(a)}_{\ \ \ \ (b)(c)}e_{(a)}$, where D is the Levi Civita connection:

In [7]:
omega_udd = Ricci_rotation_coefficents(nabla.coef(e))

Calculation of $\omega_{(a)(b)(c)}$ with the method down():

In [8]:
omega_ddd = omega_udd.down(1)

## Spinor bundle

Initialization of the spinor bundle as an abstract vector bundle, and definition of a local frame $\{e_{[sb]i}\}_{i=0,1,2,3}$ :

In [9]:
sb = M.vector_bundle(4, 'S', field='complex') #definition of the spinor bundle
e_sb = sb.local_frame('e')

Definition of Dirac's gamma matrices $\{\gamma^{(a)}\}_{(a)=0,1,2,3}$ and the identity matrix $\mathbb{1}$ :

In [10]:
Gamma_0 = sb.section_module().automorphism() #The gamma matrices are defined as automorphism of the section module of the spinor bundle
Gamma_1 = sb.section_module().automorphism()
Gamma_2 = sb.section_module().automorphism() 
Gamma_3 = sb.section_module().automorphism() 
id_s = sb.section_module().automorphism() #Identity automorphism of the section module of the spinor bundle

Gamma_0[e_sb,:] = [[I,0,0,0],[0,I,0,0],[0,0,-I,0],[0,0,0,-I]] #use I instead of i otherwise it is interpreted as an index
Gamma_1[e_sb,:] = [[0,0,0,I],[0,0,I,0],[0,-I,0,0],[-I,0,0,0]] # (-,+,+,+) signature
Gamma_2[e_sb,:] = [[0,0,0,1],[0,0,-1,0],[0,-1,0,0],[1,0,0,0]]
Gamma_3[e_sb,:] = [[0,0,I,0],[0,0,0,-I],[-I,0,0,0],[0,I,0,0]]

id_s[e_sb,:] = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]] # identity matrix

Gamma_a = (Gamma_0,Gamma_1,Gamma_2,Gamma_3) #tuple of Gamma_{(a)}

Definition of the anticommutator $\{-,-\}$ :

In [11]:
def acomm(Gamma_a,Gamma_b):
    return Gamma_a*Gamma_b+Gamma_b*Gamma_a

Check of the Clifford algebra $\{\gamma^{(a)},\gamma^{(b)}\} = 2\eta^{(a)(b)}\mathbb{1}$ :

In [12]:
for i in range(0,4): #check of Clifford algebra
    for j in range(0,4):
        if(acomm(Gamma_a[i],Gamma_a[j]) != 2*id_s*eta[i,j]):
            print("error")

## Spinor bundle connection

The connection of an arbitrary vector bundle $V$ is a map of the form $D_{V}: TM \times V \to V$ that takes a section of the tangent bundle and a section of the vecor bundle $V$ and gives a section of the vector bundle $V$.
The map $D_{V}(-,e_{[V]i})=\Gamma^{j}_{\ \ \ \ i}(-)e_{[V]j}$ ($\{e_{[V]i}\}_{i=1,2,3,4}$ local frame on $V$) defines naturally a collection of 1-forms $\Gamma^{j}_{\ \ \ \ i}$ named "connection 1-forms". In our case $V$ is the spinor bundle defined before. 

In this case the indices $i$ and $j$ are spinor indices because of course the connection acts on spinor fields (section of the spinor bundle).

Definition of the frame bundle connection 1-forms $\Gamma^{j}_{ \ \ \ \ i}$ from the Ricci rotation coefficents and Dirac's matrices as $\Gamma^{j}_{ \ \ \ \ i}= \frac{1}{4}\omega_{(a)(b)(c)}(\gamma^{(a)}\gamma^{(b)})^{j}_{ \ \ \ \ i}e^{(c)}$ :

In [13]:
nabla_sb = sb.bundle_connection('\\nabla')  #definition of the connection on the spinor bundle

par_0 = sb.section_module().automorphism()
par_1 = sb.section_module().automorphism()
par_2 = sb.section_module().automorphism()
par_3 = sb.section_module().automorphism()

par_0[e_sb,:] = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
par_1[e_sb,:] = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
par_2[e_sb,:] = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
par_3[e_sb,:] = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]

for i in range(0,4):
    for j in range(0,4):
        par_0 +=  Gamma_a[i]*Gamma_a[j]*omega_ddd.coef[i,j,0] 
        
for i in range(0,4):
    for j in range(0,4):
        par_1 +=  Gamma_a[i]*Gamma_a[j]*omega_ddd.coef[i,j,1] 
        
for i in range(0,4):
    for j in range(0,4):
        par_2 +=  Gamma_a[i]*Gamma_a[j]*omega_ddd.coef[i,j,2]      

for i in range(0,4):
    for j in range(0,4):
        par_3 +=  Gamma_a[i]*Gamma_a[j]*omega_ddd.coef[i,j,3] 

for i in range(0,4): #setting the connection 1-forms as the connection evaluated on the tetrad
    for j in range(0,4):
            nabla_sb.set_connection_form(i, j)[:] = (1/4)*(par_0[i,j]*de[0] + par_1[i,j]*de[1] + par_2[i,j]*de[2] + par_3[i,j]*de[3])


Definition of a test section of the spinor bundle $\psi = \psi^{i}e_{[s]i}$ :

In [14]:
psi1 = function("psi0", nargs = 4)
psi2 = function("psi1", nargs = 4)
psi3 = function("psi2", nargs = 4)
psi4 = function("psi3", nargs = 4)


psi1_ = M.scalar_field({BL: psi1(r,t,th,ph)}, name = '\\psi^{(1)}')
psi2_ = M.scalar_field({BL: psi2(r,t,th,ph)}, name = '\\psi^{(2)}')
psi3_ = M.scalar_field({BL: psi3(r,t,th,ph)}, name = '\\psi^{(3)}')
psi4_ = M.scalar_field({BL: psi4(r,t,th,ph)}, name = '\\psi^{(4)}')

psi = sb.section({e_sb: [psi1_,psi2_,psi3_,psi4_]}, "psi")
psi.display()

psi1(r, t, th, ph) e_0 + psi2(r, t, th, ph) e_1 + psi3(r, t, th, ph) e_2 + psi3(r, t, th, ph) e_3

Construction of the collection $\{(D_{sb}(e_{(a)},\psi))^{i} = e_{(a)}^{\ \ \ \ \mu}\psi^{i}_{\ \ \ : \mu} = e_{(a)}^{\ \ \ \ \mu}\psi^{i}_{\ \ ,\mu} + \Gamma^{i}_{ \ \ \ \ j}(e_{(a)})\psi^{j}\}_{(a)=0,1,2,3}$ as a tuple:

In [15]:
e_nab_psi = (nabla_sb(e[0],psi),nabla_sb(e[1],psi),nabla_sb(e[2],psi),nabla_sb(e[3],psi))

Calculation of $(\gamma^{(a)})^{i}_{ \ \ j}(D_{sb}(e_{(a)},\psi))^{j}$ (rhs of Dirac's equation on curved spacetime for a massless spinor field) :

In [16]:
nab_psi = sb.section( latexname='\\nabla\\psi' )
nab_psi[e_sb,:] = [0,0,0,0]

for i in range(0,4):
    nab_psi += Gamma_a[i](e_nab_psi[i]) 
    
nab_psi.display_comp()

X^0 = 1/2*((I*a^4*r - 2*I*a^2*m*r^2 + I*a^2*r^3)*psi3(r, t, th, ph)*sin(th)^3 + (2*I*a*r^4*d(psi1)/dph + (2*I*a^5*d(psi1)/dph + (-I*a^4*m + I*a^4*r)*psi3(r, t, th, ph) + (2*I*a^6 + 2*I*a^4*r^2)*d(psi1)/dt + (2*I*a^6 - 4*I*a^4*m*r + 2*I*a^4*r^2)*d(psi3)/dr)*cos(th)^4 + (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^3*psi2(r, t, th, ph) + (4*I*a^3*r^2*d(psi1)/dph + (-2*I*a^2*m*r^2 + 2*I*a^2*r^3)*psi3(r, t, th, ph) + (4*I*a^4*r^2 + 4*I*a^2*r^4)*d(psi1)/dt + (4*I*a^4*r^2 - 8*I*a^2*m*r^3 + 4*I*a^2*r^4)*d(psi3)/dr)*cos(th)^2 + (a^3*r^2 - 2*a*m*r^3 + a*r^4)*cos(th)*psi2(r, t, th, ph) + (-I*a^4*r + 2*I*a^2*m*r^2 - 2*I*a^2*r^3 + I*m*r^4)*psi3(r, t, th, ph) + (2*I*a^2*r^4 + 2*I*r^6)*d(psi1)/dt + (2*I*a^2*r^4 - 4*I*m*r^5 + 2*I*r^6)*d(psi3)/dr)*sin(th) + (2*I*a^4*cos(th)^4*d(psi3)/dph + 4*I*a^2*r^2*cos(th)^2*d(psi3)/dph + 2*I*r^4*d(psi3)/dph + (a^4 + 2*a^2*r^2 + r^4)*cos(th)*psi3(r, t, th, ph) + (2*I*a^5*cos(th)^4*d(psi3)/dt + 2*I*a*r^4*d(psi3)/dt - I*a*r^3*psi2(r, t, th, ph) + (4*I*a^3*r^2*d(psi3)/dt - I*a^3*r*psi2(r, t, th, ph))*cos(th)^2 - (a^4 + a^2*r^2)*cos(th)*psi3(r, t, th, ph))*sin(th)^2 + 2*(a^4*cos(th)^4*d(psi3)/dth + 2*a^2*r^2*cos(th)^2*d(psi3)/dth + r^4*d(psi3)/dth)*sin(th))*sqrt(a^2 - 2*m*r + r^2))/((a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)*sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*sin(th)) 
X^1 = -1/2*((-I*a^4*r + 2*I*a^2*m*r^2 - I*a^2*r^3)*psi3(r, t, th, ph)*sin(th)^3 + (-2*I*a*r^4*d(psi2)/dph + (-2*I*a^5*d(psi2)/dph + (I*a^4*m - I*a^4*r)*psi3(r, t, th, ph) + (-2*I*a^6 - 2*I*a^4*r^2)*d(psi2)/dt + (-2*I*a^6 + 4*I*a^4*m*r - 2*I*a^4*r^2)*d(psi3)/dr)*cos(th)^4 - (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^3*psi1(r, t, th, ph) + (-4*I*a^3*r^2*d(psi2)/dph + (2*I*a^2*m*r^2 - 2*I*a^2*r^3)*psi3(r, t, th, ph) + (-4*I*a^4*r^2 - 4*I*a^2*r^4)*d(psi2)/dt + (-4*I*a^4*r^2 + 8*I*a^2*m*r^3 - 4*I*a^2*r^4)*d(psi3)/dr)*cos(th)^2 - (a^3*r^2 - 2*a*m*r^3 + a*r^4)*cos(th)*psi1(r, t, th, ph) + (I*a^4*r - 2*I*a^2*m*r^2 + 2*I*a^2*r^3 - I*m*r^4)*psi3(r, t, th, ph) + (-2*I*a^2*r^4 - 2*I*r^6)*d(psi2)/dt + (-2*I*a^2*r^4 + 4*I*m*r^5 - 2*I*r^6)*d(psi3)/dr)*sin(th) + (2*I*a^4*cos(th)^4*d(psi3)/dph + 4*I*a^2*r^2*cos(th)^2*d(psi3)/dph + 2*I*r^4*d(psi3)/dph + (a^4 + 2*a^2*r^2 + r^4)*cos(th)*psi3(r, t, th, ph) + (2*I*a^5*cos(th)^4*d(psi3)/dt + 2*I*a*r^4*d(psi3)/dt - I*a*r^3*psi1(r, t, th, ph) + (4*I*a^3*r^2*d(psi3)/dt - I*a^3*r*psi1(r, t, th, ph))*cos(th)^2 - (a^4 + a^2*r^2)*cos(th)*psi3(r, t, th, ph))*sin(th)^2 + 2*(a^4*cos(th)^4*d(psi3)/dth + 2*a^2*r^2*cos(th)^2*d(psi3)/dth + r^4*d(psi3)/dth)*sin(th))*sqrt(a^2 - 2*m*r + r^2))/((a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)*sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*sin(th)) 
X^2 = -1/2*((I*a^4*r - 2*I*a^2*m*r^2 + I*a^2*r^3)*psi2(r, t, th, ph)*sin(th)^3 + (2*I*a*r^4*d(psi3)/dph + (2*I*a^5*d(psi3)/dph + (-I*a^4*m + I*a^4*r)*psi2(r, t, th, ph) + (2*I*a^6 - 4*I*a^4*m*r + 2*I*a^4*r^2)*d(psi2)/dr + (2*I*a^6 + 2*I*a^4*r^2)*d(psi3)/dt)*cos(th)^4 + (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^3*psi3(r, t, th, ph) + (4*I*a^3*r^2*d(psi3)/dph + (-2*I*a^2*m*r^2 + 2*I*a^2*r^3)*psi2(r, t, th, ph) + (4*I*a^4*r^2 - 8*I*a^2*m*r^3 + 4*I*a^2*r^4)*d(psi2)/dr + (4*I*a^4*r^2 + 4*I*a^2*r^4)*d(psi3)/dt)*cos(th)^2 + (a^3*r^2 - 2*a*m*r^3 + a*r^4)*cos(th)*psi3(r, t, th, ph) + (-I*a^4*r + 2*I*a^2*m*r^2 - 2*I*a^2*r^3 + I*m*r^4)*psi2(r, t, th, ph) + (2*I*a^2*r^4 - 4*I*m*r^5 + 2*I*r^6)*d(psi2)/dr + (2*I*a^2*r^4 + 2*I*r^6)*d(psi3)/dt)*sin(th) + (2*I*a^4*cos(th)^4*d(psi1)/dph + 4*I*a^2*r^2*cos(th)^2*d(psi1)/dph + 2*I*r^4*d(psi1)/dph + (a^4 + 2*a^2*r^2 + r^4)*cos(th)*psi2(r, t, th, ph) + (2*I*a^5*cos(th)^4*d(psi1)/dt + 2*I*a*r^4*d(psi1)/dt - I*a*r^3*psi3(r, t, th, ph) + (4*I*a^3*r^2*d(psi1)/dt - I*a^3*r*psi3(r, t, th, ph))*cos(th)^2 - (a^4 + a^2*r^2)*cos(th)*psi2(r, t, th, ph))*sin(th)^2 + 2*(a^4*cos(th)^4*d(psi2)/dth + 2*a^2*r^2*cos(th)^2*d(psi2)/dth + r^4*d(psi2)/dth)*sin(th))*sqrt(a^2 - 2*m*r + r^2))/((a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)*sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*sin(th)) 
X^3 = 1/2*((-I*a^4*r + 2*I*a^2*m*r^2 - I*a^2*r^3)*ps

## Spinor connection with the Wald field

Definition of the Wald field solution $A = \frac{B_{0}}{2}(\psi + 2a\eta)$, where $\eta$ and $\psi$ are the 1-forms naturally associated with musical isomorphism respectively to the two Killing vector fields $\frac{\partial}{\partial t}$ and $\frac{\partial}{\partial \phi}$ :

In [17]:
var('B_0')
A_wald = (B_0/2)*(BL.frame()[3].down(g)+(2*a)*BL.frame()[0].down(g))
A_wald.display()

(B_0*a*m*r - B_0*a*r^2 - (B_0*a^3 - B_0*a*m*r)*cos(th)^2)/(a^2*cos(th)^2 + r^2) dt + 1/2*(2*B_0*a^2*m*r*sin(th)^4 - (4*B_0*a^2*m*r - B_0*a^2*r^2 - B_0*r^4 - (B_0*a^4 + B_0*a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2) dph

Definition of the frame bundle connection 1-forms $(\Gamma^{(A)})^{j}_{ \ \ \ \ i}$ from the Ricci rotation coefficents as $(\Gamma^{(A)})^{j}_{ \ \ \ \ i}= \Gamma^{j}_{ \ \ \ \ i} +qA(\mathbb{1})^{j}_{ \ \ \ \ i}$ ($q$ electric charge of the particle):

In [18]:
var('q') #definition of electric charge
nabla_s_A = sb.bundle_connection('\\nabla_{A}') #definition of the new connection on the spinor bundle with A
for i in range(0,4):
    for j in range(0,4):
        nabla_s_A.set_connection_form(i, j)[:] = nabla_sb[i,j]+q*id_s[i,j]*A_wald

Construction of the collection $\{(D_{sb}^{(A)}(e_{(a)},\psi))^{i} = e_{(a)}^{\ \ \ \ \mu}(\psi^{i}_{\ \ \ : \mu}+qA_{\mu}\psi^{i}) = e_{(a)}^{\ \ \ \ \mu}\psi^{i}_{\ \ ,\mu} + (\Gamma^{(A)})^{i}_{ \ \ \ \ j}(e_{(a)})\psi^{j}\}_{(a)=0,1,2,3}$ as a tuple:

In [19]:
e_nab_A_psi = (nabla_s_A(e[0],psi),nabla_s_A(e[1],psi),nabla_s_A(e[2],psi),nabla_s_A(e[3],psi))

Calculation of $(\gamma^{(a)})^{i}_{ \ \ j}(D_{sb}^{(A)}(e_{(a)},\psi))^{j}$ (rhs of Dirac's equation on curved spacetime for a massless spinor field coupled to Wald solution $A$) :

In [20]:
nab_A_psi = sb.section( latexname='\\nabla\\psi' )
nab_A_psi[e_sb,:] = [0,0,0,0]

for i in range(0,4):
    nab_A_psi += Gamma_a[i](e_nab_A_psi[i]) 
    
nab_A_psi.display_comp()

X^0 = 1/2*((I*a^4*r - 2*I*a^2*m*r^2 + I*a^2*r^3)*psi3(r, t, th, ph)*sin(th)^3 + ((-I*B_0*a^7*q + 2*I*B_0*a^5*m*q*r - I*B_0*a^5*q*r^2)*cos(th)^6*psi1(r, t, th, ph) + 2*I*a*r^4*d(psi1)/dph + (2*I*a^5*d(psi1)/dph + (-I*B_0*a^7*q + 2*I*B_0*a^5*m*q*r - 3*I*B_0*a^5*q*r^2 + 4*I*B_0*a^3*m*q*r^3 - 2*I*B_0*a^3*q*r^4)*psi1(r, t, th, ph) + (-I*a^4*m + I*a^4*r)*psi3(r, t, th, ph) + (2*I*a^6 + 2*I*a^4*r^2)*d(psi1)/dt + (2*I*a^6 - 4*I*a^4*m*r + 2*I*a^4*r^2)*d(psi3)/dr)*cos(th)^4 + (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^3*psi2(r, t, th, ph) + (4*I*a^3*r^2*d(psi1)/dph + (-2*I*B_0*a^5*q*r^2 + 4*I*B_0*a^3*m*q*r^3 - 3*I*B_0*a^3*q*r^4 + 2*I*B_0*a*m*q*r^5 - I*B_0*a*q*r^6)*psi1(r, t, th, ph) + (-2*I*a^2*m*r^2 + 2*I*a^2*r^3)*psi3(r, t, th, ph) + (4*I*a^4*r^2 + 4*I*a^2*r^4)*d(psi1)/dt + (4*I*a^4*r^2 - 8*I*a^2*m*r^3 + 4*I*a^2*r^4)*d(psi3)/dr)*cos(th)^2 + (a^3*r^2 - 2*a*m*r^3 + a*r^4)*cos(th)*psi2(r, t, th, ph) + (-I*B_0*a^3*q*r^4 + 2*I*B_0*a*m*q*r^5 - I*B_0*a*q*r^6)*psi1(r, t, th, ph) + (-I*a^4*r + 2*I*a^2*m*r^2 - 2*I*a^2*r^3 + I*m*r^4)*psi3(r, t, th, ph) + (2*I*a^2*r^4 + 2*I*r^6)*d(psi1)/dt + (2*I*a^2*r^4 - 4*I*m*r^5 + 2*I*r^6)*d(psi3)/dr)*sin(th) + (2*I*a^4*cos(th)^4*d(psi3)/dph + 4*I*a^2*r^2*cos(th)^2*d(psi3)/dph + 2*I*r^4*d(psi3)/dph + (a^4 + 2*a^2*r^2 + r^4)*cos(th)*psi3(r, t, th, ph) + (2*I*a*r^4*d(psi3)/dt + (2*I*a^5*d(psi3)/dt + (-I*B_0*a^6*q + I*B_0*a^4*q*r^2)*psi3(r, t, th, ph))*cos(th)^4 - I*a*r^3*psi2(r, t, th, ph) + (4*I*a^3*r^2*d(psi3)/dt - I*a^3*r*psi2(r, t, th, ph) + (-2*I*B_0*a^4*q*r^2 + 2*I*B_0*a^2*q*r^4)*psi3(r, t, th, ph))*cos(th)^2 - (a^4 + a^2*r^2)*cos(th)*psi3(r, t, th, ph) + (-I*B_0*a^2*q*r^4 + I*B_0*q*r^6)*psi3(r, t, th, ph))*sin(th)^2 + 2*(a^4*cos(th)^4*d(psi3)/dth + 2*a^2*r^2*cos(th)^2*d(psi3)/dth + r^4*d(psi3)/dth)*sin(th))*sqrt(a^2 - 2*m*r + r^2))/((a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)*sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*sin(th)) 
X^1 = -1/2*((-I*a^4*r + 2*I*a^2*m*r^2 - I*a^2*r^3)*psi3(r, t, th, ph)*sin(th)^3 + ((I*B_0*a^7*q - 2*I*B_0*a^5*m*q*r + I*B_0*a^5*q*r^2)*cos(th)^6*psi2(r, t, th, ph) - 2*I*a*r^4*d(psi2)/dph + (-2*I*a^5*d(psi2)/dph + (I*B_0*a^7*q - 2*I*B_0*a^5*m*q*r + 3*I*B_0*a^5*q*r^2 - 4*I*B_0*a^3*m*q*r^3 + 2*I*B_0*a^3*q*r^4)*psi2(r, t, th, ph) + (I*a^4*m - I*a^4*r)*psi3(r, t, th, ph) + (-2*I*a^6 - 2*I*a^4*r^2)*d(psi2)/dt + (-2*I*a^6 + 4*I*a^4*m*r - 2*I*a^4*r^2)*d(psi3)/dr)*cos(th)^4 - (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^3*psi1(r, t, th, ph) + (-4*I*a^3*r^2*d(psi2)/dph + (2*I*B_0*a^5*q*r^2 - 4*I*B_0*a^3*m*q*r^3 + 3*I*B_0*a^3*q*r^4 - 2*I*B_0*a*m*q*r^5 + I*B_0*a*q*r^6)*psi2(r, t, th, ph) + (2*I*a^2*m*r^2 - 2*I*a^2*r^3)*psi3(r, t, th, ph) + (-4*I*a^4*r^2 - 4*I*a^2*r^4)*d(psi2)/dt + (-4*I*a^4*r^2 + 8*I*a^2*m*r^3 - 4*I*a^2*r^4)*d(psi3)/dr)*cos(th)^2 - (a^3*r^2 - 2*a*m*r^3 + a*r^4)*cos(th)*psi1(r, t, th, ph) + (I*B_0*a^3*q*r^4 - 2*I*B_0*a*m*q*r^5 + I*B_0*a*q*r^6)*psi2(r, t, th, ph) + (I*a^4*r - 2*I*a^2*m*r^2 + 2*I*a^2*r^3 - I*m*r^4)*psi3(r, t, th, ph) + (-2*I*a^2*r^4 - 2*I*r^6)*d(psi2)/dt + (-2*I*a^2*r^4 + 4*I*m*r^5 - 2*I*r^6)*d(psi3)/dr)*sin(th) + (2*I*a^4*cos(th)^4*d(psi3)/dph + 4*I*a^2*r^2*cos(th)^2*d(psi3)/dph + 2*I*r^4*d(psi3)/dph + (a^4 + 2*a^2*r^2 + r^4)*cos(th)*psi3(r, t, th, ph) + (2*I*a*r^4*d(psi3)/dt + (2*I*a^5*d(psi3)/dt + (-I*B_0*a^6*q + I*B_0*a^4*q*r^2)*psi3(r, t, th, ph))*cos(th)^4 - I*a*r^3*psi1(r, t, th, ph) + (4*I*a^3*r^2*d(psi3)/dt - I*a^3*r*psi1(r, t, th, ph) + (-2*I*B_0*a^4*q*r^2 + 2*I*B_0*a^2*q*r^4)*psi3(r, t, th, ph))*cos(th)^2 - (a^4 + a^2*r^2)*cos(th)*psi3(r, t, th, ph) + (-I*B_0*a^2*q*r^4 + I*B_0*q*r^6)*psi3(r, t, th, ph))*sin(th)^2 + 2*(a^4*cos(th)^4*d(psi3)/dth + 2*a^2*r^2*cos(th)^2*d(psi3)/dth + r^4*d(psi3)/dth)*sin(th))*sqrt(a^2 - 2*m*r + r^2))/((a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)*sqrt(a^2*cos(th)^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*sin(th)) 
X^2 = -1/2*((I*a^4*r - 2*I*a^2*m*r^2 + I*a^2*r^3)*psi2(r, t, th, ph)*sin(th)^3 + ((-I*B_0*a^7*q + 2*I*B_0*a^5*m*q*r - I*B_0*a^5*q*r^2)*cos(th)^6*psi3(r, t, th, ph) + 2*I*a*r^4*d(psi3)/dph + (2*I*a^5*d(psi3)/dph + (-I*a^4*m + I*a^4*